## Tight binding model

## 1d mono-atomic chain
Here we show the electronic structure of 1d atomic chain as an example of tight binding model. In each unitcell, there is one atom, with an s orbital. Between each two first nearest neighbors, the hopping of the two s orbitals is t. This system has the energy dispertion of $E_k=2t cos(k)$.

In [1]:
%matplotlib inline
import numpy as np
from  minimulti.electron.Hamiltonian import etb_model, atoms_model
from minimulti.electron.basis2 import BasisSet, gen_basis_set, atoms_to_basis
from ase.atoms import Atoms
import matplotlib.pyplot as plt


def run_1d_chain_1atom(t=1, nel=1):
    # generate structure.
    atoms=Atoms(symbols='H', positions=[(0,0,0)], cell=[1,1,1])
    # generate basis set
    bset= atoms_to_basis(atoms, basis_dict={'H': ('s',)}, nspin=2)
    # generate model
    mymodel=atoms_model(atoms=atoms, basis_set=bset, nspin=2)

    # onsite energy (tune v to make electron localized on one site)
    #mymodel.set_onsite(v,0)
    #mymodel.set_onsite(v,1)

    # 1D hopping along z. Format: t, i, j, R.
    # spin up
    mymodel.set_hop(t, 0,0, [0,0,1])
    # spin down
    mymodel.set_hop(t, 1,1, [0,0,1])
    
        # spin up
    mymodel.set_hop(t, 0,0, [0,1,0])
    # spin down
    mymodel.set_hop(t, 1,1, [0,1,0])

    mymodel.set(nel=nel, mixing=0.5, tol_energy=1e-8, tol_rho=1e-8, sigma=0.05)
    mymodel.set_Hubbard_U(Utype='Dudarev', Hubbard_dict={'H':{'U':0, 'J':0}})

    # K mesh.
    mymodel.set_kmesh([1,8,30])

    # Scf solve
    mymodel.scf_solve(print_iter_info=True)
    efermi=mymodel.get_fermi_level()

    kpt_x=np.arange(0, 1.01,0.01)
    kpts=[np.array([0,0,1]) * x for x in kpt_x]
    evalues, evecs=mymodel.solve_all(k_list=kpts)

    for i in range(mymodel._norb):
        plt.plot(kpt_x, evalues[i,:])
    plt.xlabel('k-point')
    plt.ylabel('Energy')
    plt.axhline(efermi, linestyle='--', color='gray')

In [2]:
from ipywidgets import FloatSlider, interact_manual
#run_1d_chain_1atom(t=1, nel=1)
interact_manual(run_1d_chain_1atom, 
                t=FloatSlider(min=0,max=1, step=0.1,value=1.0),  
                nel=FloatSlider(min=0,max=2, step=0.1,value=1.0 ));

interactive(children=(FloatSlider(value=1.0, description='t', max=1.0), FloatSlider(value=1.0, description='ne…

In [3]:
%pylab inline
import numpy as np
from  minimulti.electron.Hamiltonian import etb_model, atoms_model
from minimulti.electron.basis2 import BasisSet, gen_basis_set, atoms_to_basis
from ase.atoms import Atoms
import matplotlib.pyplot as plt


def run_1d_chain_2atom(t=1,  dt=0.0, nel=2):
    # generate structure.
    atoms=Atoms(symbols='H', positions=[(0,0,0)], cell=[1,1,1])
    # generate basis set
    bset= atoms_to_basis(atoms, basis_dict={'H': ('s',)}, nspin=2)
    # generate model
    mymodel=atoms_model(atoms=atoms, basis_set=bset, nspin=2)

    # make supercell
    sc_matrix=np.diag([1,1,2])
    nsc=np.linalg.det(sc_matrix)
    mymodel=mymodel.make_supercell(sc_matrix)
    # onsite energy (tune v to make electron localized on one site)
    # mymodel.set_onsite(v,0)
    # mymodel.set_onsite(v,1)

    # 1D hopping along z. Format: t, i, j, R.
    # spin up
    mymodel.set_hop(t-dt, 0,2, [0,0,0])
    mymodel.set_hop(t+dt, 2,0,[0,0,1])
    # spin down
    mymodel.set_hop(t-dt, 1,3, [0,0,0])
    mymodel.set_hop(t+dt, 3,1, [0,0,1])
    mymodel.set(nel=nel*nsc, mixing=0.5, tol_energy=1e-8, tol_rho=1e-8, sigma=0.1)
    mymodel.set_Hubbard_U(Utype='Dudarev', Hubbard_dict={'H':{'U':0, 'J':0}})

    # K mesh.
    mymodel.set_kmesh([1,1,30])

    # Scf solve
    mymodel.scf_solve(print_iter_info=False)
    efermi=mymodel.get_fermi_level()

    kpt_x=np.arange(0, 1.01,0.01)
    kpts=[np.array([0,0,1]) * x for x in kpt_x]
    evalues, evecs=mymodel.solve_all(k_list=kpts)

    for i in range(mymodel._norb):
        plt.plot(kpt_x, evalues[i,:], color='black')
    plt.xlabel('k-point')
    plt.ylabel('Energy')
    plt.axhline(efermi, linestyle='--', color='gray')

Populating the interactive namespace from numpy and matplotlib


The simple 1d-chain can provide an example of how an insulator can be formed by cell-doubling. Let's consider the same 1d-chain, but this time, we use two unit of atoms as our cell. This would reduce the Brillouin zone to half its size. (The unit of BZ is $2\pi/2a$ instead of $2\pi/a$ in the following figure.) Now let one atom move toward
Of cource, if nothing is changed, the property of the system is the same as before: it is metallic. 

<img src="electron_1d.png" width="400">

In [4]:
from ipywidgets import FloatSlider, interact, interact_manual, fixed
#run_1d_chain_1atom(t=1, nel=1)
interact(run_1d_chain_2atom, 
                t=fixed(1.0),
                dt=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.0),
                nel=fixed(1.0));

interactive(children=(FloatSlider(value=0.0, description='dt', max=1.0), Output()), _dom_classes=('widget-inte…

In [5]:
from ipywidgets import FloatSlider, interact, interact_manual, fixed
#run_1d_chain_1atom(t=1, nel=1)
interact(run_1d_chain_2atom, 
                t=fixed(1.0),
                dt=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.3),
                nel=fixed(1.0));

interactive(children=(FloatSlider(value=0.3, description='dt', max=1.0), Output()), _dom_classes=('widget-inte…